In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('df.csv')

In [3]:
df.head()

,Unnamed: 0,priority,mark,kvota,vstup,mark_mean_years,placesCount_years,budgetCount_years,contractCount_years,budget_mark_mean_years,abit_count_years,mark_max_years,mark_min_years,budget_mark_max_years,budget_mark_min_years
0,0,3,0.995548,0,0,0.746759,0.042491,0.051154,1.143596e-08,0.991546,0.157225,1.0,0.585326,0.90961,1.0
1,1,0,0.986195,0,1,0.746759,0.042491,0.051154,1.143596e-08,0.991546,0.157225,1.0,0.585326,0.90961,1.0
2,2,9,0.982558,0,0,0.746759,0.042491,0.051154,1.143596e-08,0.991546,0.157225,1.0,0.585326,0.90961,1.0
3,3,2,0.977881,0,0,0.746759,0.042491,0.051154,1.143596e-08,0.991546,0.157225,1.0,0.585326,0.90961,1.0
4,4,3,0.974764,0,0,0.746759,0.042491,0.051154,1.143596e-08,0.991546,0.157225,1.0,0.585326,0.90961,1.0


In [4]:
df = df.drop('Unnamed: 0', axis = 1)

In [5]:
df.groupby('vstup').vstup.count()

vstup
0    186590
1     18389
Name: vstup, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('vstup', axis = 1), df.vstup, test_size = 10000)

In [8]:
from imblearn.over_sampling import SMOTE

In [9]:
from collections import Counter

In [10]:
smote = SMOTE()

In [11]:
X_up, y_up = smote.fit_resample(X_train, y_train)

In [12]:
counter = Counter(y_up)

In [13]:
counter

Counter({0: 177513, 1: 177513})

In [14]:
X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_up, y_up, test_size = 10000, random_state = 42)

In [15]:
from imblearn.under_sampling import RandomUnderSampler

In [16]:
RUS = RandomUnderSampler()

In [17]:
X_down, y_down = RUS.fit_resample(X_train, y_train)

In [18]:
counter = Counter(y_down)
print(counter)

Counter({0: 17466, 1: 17466})


In [19]:
X_train_down, X_test_down, y_train_down, y_test_down = train_test_split(X_down, y_down, test_size = 0.1, random_state = 42)

In [20]:
y_train.count()

194979

In [21]:
y_train_up.count()

345026

In [22]:
y_test_up.count()

10000

In [23]:
y_train_down.count()

31438

In [24]:
y_test_down.count()

3494

In [25]:
from imblearn.combine import SMOTETomek

In [26]:
smoteT = SMOTETomek()

In [27]:
X_mid, y_mid = smoteT.fit_resample(X_train, y_train)

In [28]:
counter = Counter(y_mid)
print(counter)

Counter({0: 170255, 1: 170255})


In [29]:
X_train_mid, X_test_mid, y_train_mid, y_test_mid = train_test_split(X_mid, y_mid, test_size = 10000, random_state = 42)

In [30]:
y_train_up.count()

345026

In [31]:
y_train_mid.count()

330510

In [32]:
y_train_down.count()

31438

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
log1 = LogisticRegression(random_state = 42, max_iter = 1000000)
log1.fit(X_train_up, y_train_up)
print(log1.score(X_train_up, y_train_up))
print(log1.score(X_test_up, y_test_up))

0.8311402618933066
0.8289


In [35]:
log2 = LogisticRegression(random_state = 42, max_iter = 1000000)
log2.fit(X_train_mid, y_train_mid)
print(log2.score(X_train_mid, y_train_mid))
print(log2.score(X_test_mid, y_test_mid))

0.8459441469244501
0.8491


In [36]:
log2.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [37]:
log2.coef_

array([[ -0.76109142,   8.8153032 ,   3.0081087 ,   1.36604374,
         -0.91553022,   1.17992122,  -0.01270313, -10.41194626,
         -0.13509675,  -0.75013827,  -0.44192249,   1.65424754,
          0.58076516]])

In [38]:
X_train_mid.columns

Index(['priority', 'mark', 'kvota', 'mark_mean_years', 'placesCount_years',
       'budgetCount_years', 'contractCount_years', 'budget_mark_mean_years',
       'abit_count_years', 'mark_max_years', 'mark_min_years',
       'budget_mark_max_years', 'budget_mark_min_years'],
      dtype='object')

In [39]:
log2.intercept_ 

array([0.56197568])

In [40]:
log = LogisticRegression(random_state = 42, max_iter = 1000000)
log.fit(X_train_mid, y_train_mid)
print(log.score(X_train_mid, y_train_mid))
print(log.score(X_test_mid, y_test_mid))

0.8459441469244501
0.8491


In [41]:
from sklearn.tree import DecisionTreeClassifier

In [42]:
tree1 = DecisionTreeClassifier()
tree1.fit(X_train_up, y_train_up)
print(tree1.score(X_train_up, y_train_up))
print(tree1.score(X_test_up, y_test_up))

0.9934149890153205
0.9109


In [43]:
tree2 = DecisionTreeClassifier()
tree2.fit(X_train_mid, y_train_mid)
print(tree2.score(X_train_mid, y_train_mid))
print(tree2.score(X_test_mid, y_test_mid))

0.9930773652839551
0.9596


In [44]:
from sklearn.svm import SVC

In [45]:
svc = SVC(random_state = 42)
svc.fit(X_train_down, y_train_down)
print(svc.score(X_train_down, y_train_down))
print(svc.score(X_test_down, y_test_down))
print(svc.score(X_test_up, y_test_up))

0.8349767796933647
0.8423010875787064
0.836


In [46]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
GNB = GaussianNB()
MNB = MultinomialNB()
BNB = BernoulliNB()
CNB = CategoricalNB()
GNB.fit(X_train_up, y_train_up)
MNB.fit(X_train_up, y_train_up)
BNB.fit(X_train_up, y_train_up)
CNB.fit(X_train_up, y_train_up)
print('GNB train:', GNB.score(X_train_up, y_train_up))
print('GNB test:', GNB.score(X_test_up, y_test_up))
print()
print('MNB train:', MNB.score(X_train_up, y_train_up))
print('MNB test:', MNB.score(X_test_up, y_test_up))
print()
print('BNB train:', BNB.score(X_train_up, y_train_up))
print('BNB test:', BNB.score(X_test_up, y_test_up))
print()
print('CNB train:', CNB.score(X_train_up, y_train_up))
print('CNB test:', CNB.score(X_test_up, y_test_up))

GNB train: 0.7023760528192078
GNB test: 0.7001

MNB train: 0.7552329389669185
MNB test: 0.7532

BNB train: 0.7414049955655516
BNB test: 0.7358

CNB train: 0.755415533901793
CNB test: 0.7494


In [47]:
tree = DecisionTreeClassifier()
tree.fit(X_train_mid, y_train_mid)
print(tree.score(X_train_mid, y_train_mid))
print(tree.score(X_test_mid, y_test_mid))

0.9930773652839551
0.9596


In [48]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [49]:
def ModelMetrics(model):
    print('accuracy:', accuracy_score(model.predict(X_test_mid), y_test_mid))
    print('f1_score:', f1_score(model.predict(X_test_mid), y_test_mid))
    print('roc_auc_score:', roc_auc_score(model.predict(X_test_mid), y_test_mid))
    print('precision_score:', precision_score(model.predict(X_test_mid), y_test_mid))
    print('recall_score:', recall_score(model.predict(X_test_mid), y_test_mid))
    bias = 1 - accuracy_score(model.predict(X_train_mid), y_train_mid)
    print('bias = ', bias)
    variance = 1 - accuracy_score(model.predict(X_test_mid), y_test_mid) - bias
    print('variance = ', variance)

In [50]:
from sklearn.ensemble import StackingClassifier

In [51]:
estimators = [
    ('log', log),\
    ('svc', svc),\
    ('mnb', MNB),\
]

In [52]:
clf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(),\
                        n_jobs = -1, passthrough = True)

In [53]:
clf.fit(X_train_down, y_train_down)

StackingClassifier(estimators=[('log',
                                LogisticRegression(max_iter=1000000,
                                                   random_state=42)),
                               ('svc', SVC(random_state=42)),
                               ('mnb', MultinomialNB())],
                   final_estimator=DecisionTreeClassifier(), n_jobs=-1,
                   passthrough=True)

In [54]:
print(clf.score(X_train_mid, y_train_mid))
ModelMetrics(clf)

0.889897431242625
accuracy: 0.8921
f1_score: 0.8927328760314146
roc_auc_score: 0.8923389355742296
precision_score: 0.9054244807420851
recall_score: 0.8803921568627451
bias =  0.11010256875737501
variance =  -0.002202568757375012


In [55]:
tree = DecisionTreeClassifier(random_state = 42)
tree.fit(X_train_mid, y_train_mid)
print(tree.score(X_train_mid, y_train_mid))
print(tree.score(X_test_mid, y_test_mid))
ModelMetrics(tree)

0.9930773652839551
0.9594
accuracy: 0.9594
f1_score: 0.958890238963143
roc_auc_score: 0.9594585483975964
precision_score: 0.9548296027424884
recall_score: 0.9629855603009966
bias =  0.006922634716044929
variance =  0.03367736528395504


In [56]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
forest = RandomForestClassifier(random_state = 42)
forest.fit(X_train_mid, y_train_mid)
print(forest.score(X_train_mid, y_train_mid))
print(forest.score(X_test_mid, y_test_mid))
ModelMetrics(forest)

0.9930531602674655
0.9637
accuracy: 0.9637
f1_score: 0.9634109464771696
roc_auc_score: 0.9636956230591879
precision_score: 0.9637023593466425
recall_score: 0.9631197097944377
bias =  0.0069468397325345155
variance =  0.029353160267465483


In [58]:
from sklearn.ensemble import GradientBoostingClassifier

In [59]:
GBS = GradientBoostingClassifier(random_state = 42)
GBS.fit(X_train_mid, y_train_mid)
print(GBS.score(X_train_mid, y_train_mid))
print(GBS.score(X_test_mid, y_test_mid))
ModelMetrics(GBS)

0.8847780702550604
0.886
accuracy: 0.886
f1_score: 0.8878394332939786
roc_auc_score: 0.8868183416322839
precision_score: 0.9098608590441621
recall_score: 0.8668587896253602
bias =  0.11522192974493961
variance =  -0.0012219297449396205


In [60]:
from sklearn.ensemble import AdaBoostClassifier

In [61]:
Ada = AdaBoostClassifier(tree, random_state = 42)
Ada.fit(X_train_mid, y_train_mid)
print(Ada.score(X_train_mid, y_train_mid))
print(Ada.score(X_test_mid, y_test_mid))
ModelMetrics(Ada)

0.9930773652839551
0.9618
accuracy: 0.9618
f1_score: 0.9615539452495975
roc_auc_score: 0.9617909114956872
precision_score: 0.9632990522282718
recall_score: 0.9598151496885674
bias =  0.006922634716044929
variance =  0.03127736528395508


In [62]:
params = {'max_depth' : [None, 20, 50, 75],\
         'n_estimators' : [50, 100, 200, 250, 400]}

In [63]:
from sklearn.model_selection import GridSearchCV

In [64]:
forestGSCV = GridSearchCV(forest, params)
forestGSCV.fit(X_train_mid, y_train_mid)
print(forestGSCV.best_params_)
bestForest = forestGSCV.best_estimator_
print(bestForest.score(X_train_mid, y_train_mid))
print(bestForest.score(X_test_mid, y_test_mid))
ModelMetrics(bestForest)

{'max_depth': None, 'n_estimators': 400}
0.9930773652839551
0.9638
accuracy: 0.9638
f1_score: 0.9635301229095306
roc_auc_score: 0.9637931428293016
precision_score: 0.9643073200241984
recall_score: 0.962754177571975
bias =  0.006922634716044929
variance =  0.02927736528395508


In [65]:
params = {'n_estimators' : [250, 300, 350]}

In [66]:
from sklearn.model_selection import GridSearchCV

In [67]:
forestGSCV = GridSearchCV(forest, params)
forestGSCV.fit(X_train_mid, y_train_mid)
print(forestGSCV.best_params_)
bestForest = forestGSCV.best_estimator_
print(bestForest.score(X_train_mid, y_train_mid))
print(bestForest.score(X_test_mid, y_test_mid))
ModelMetrics(bestForest)

{'n_estimators': 300}
0.9930773652839551
0.9637
accuracy: 0.9637
f1_score: 0.9634330613478391
roc_auc_score: 0.963692752855157
precision_score: 0.9643073200241984
recall_score: 0.9625603864734299
bias =  0.006922634716044929
variance =  0.02937736528395507


In [68]:
params = {'n_estimators' : [50, 100, 200, 250, 400]}

In [69]:
AdaGSCV = GridSearchCV(Ada, params)
AdaGSCV.fit(X_train_mid, y_train_mid)
print(AdaGSCV.best_params_)
bestAda = AdaGSCV.best_estimator_
print(bestAda.score(X_train_mid, y_train_mid))
print(bestAda.score(X_test_mid, y_test_mid))
ModelMetrics(bestAda)

{'n_estimators': 100}
0.9930773652839551
0.9622
accuracy: 0.9622
f1_score: 0.9619641779029986
roc_auc_score: 0.9621909330480589
precision_score: 0.9639040129058278
recall_score: 0.9600321349668608
bias =  0.006922634716044929
variance =  0.030877365283955016


In [70]:
y_test_mid.count()

10000

In [71]:
from joblib import dump, load

In [72]:
dump(forestGSCV.best_estimator_, 'forestGSCV.joblib')

['forestGSCV.joblib']

In [73]:
X_train_mid.columns

Index(['priority', 'mark', 'kvota', 'mark_mean_years', 'placesCount_years',
       'budgetCount_years', 'contractCount_years', 'budget_mark_mean_years',
       'abit_count_years', 'mark_max_years', 'mark_min_years',
       'budget_mark_max_years', 'budget_mark_min_years'],
      dtype='object')

In [74]:
from sklearn.feature_selection import RFECV

In [75]:
rfeForest = RFECV(forest, min_features_to_select = 5)

In [76]:
rfeForest.fit(X_train_mid, y_train_mid)

RFECV(estimator=RandomForestClassifier(random_state=42),
      min_features_to_select=5)

In [77]:
rfeForest.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [78]:
rfeForest.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [79]:
X_train_sub = rfeForest.transform(X_train_mid)
X_test_sub = rfeForest.transform(X_test_mid)

In [80]:
forestSub = RandomForestClassifier(random_state = 42, n_estimators = 250)
forestSub.fit(X_train_sub, y_train_mid)
print(forestSub.score(X_train_sub, y_train_mid))
print(forestSub.score(X_test_sub, y_test_mid))

0.9930773652839551
0.9637


In [81]:
print('accuracy:', accuracy_score(forestSub.predict(X_test_sub), y_test_mid))
print('f1_score:', f1_score(forestSub.predict(X_test_sub), y_test_mid))
print('roc_auc_score:', roc_auc_score(forestSub.predict(X_test_sub), y_test_mid))
print('precision_score:', precision_score(forestSub.predict(X_test_sub), y_test_mid))
print('recall_score:', recall_score(forestSub.predict(X_test_sub), y_test_mid))
bias = 1 - accuracy_score(forestSub.predict(X_train_sub), y_train_mid)
print('bias = ', bias)
variance = 1 - accuracy_score(forestSub.predict(X_test_sub), y_test_mid) - bias
print('variance = ', variance)

accuracy: 0.9637
f1_score: 0.9634404270319266
roc_auc_score: 0.963692092915345
precision_score: 0.9645089735833837
recall_score: 0.962374245472837
bias =  0.006922634716044929
variance =  0.02937736528395507


In [82]:
params = {
    'min_samples_split' : [2, 50, 100, 250, 1000],\
    'min_samples_leaf' : [1, 50, 100, 250, 1000], \
    'min_weight_fraction_leaf' : [0.0, 0.25, 0.5],\
    'max_leaf_nodes' : [None, 2, 3, 5, 10],\
    'min_impurity_decrease' : [0.0, 0.25, 0.5, 0.75],\
    'max_depth' : [None, 25, 50, 75, 12]
}

In [83]:
from sklearn.model_selection import RandomizedSearchCV

In [84]:
GSCV = RandomizedSearchCV(forestSub, params, n_iter = 250, verbose = 3, return_train_score = True)

In [85]:
GSCV.fit(X_train_sub, y_train_mid)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV 1/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.2s
[CV 2/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.3s
[CV 3/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.3s
[CV 4/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.1s
[CV 5/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=50, min_samples_split=100, min_weight_fractio

[CV 2/5] END max_depth=12, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=50, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.821, test=0.822) total time=  23.9s
[CV 3/5] END max_depth=12, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=50, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.821, test=0.822) total time=  23.8s
[CV 4/5] END max_depth=12, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=50, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.822, test=0.820) total time=  23.7s
[CV 5/5] END max_depth=12, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=50, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.820, test=0.823) total time=  23.8s
[CV 1/5] END max_depth=75, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1000, min_samples_split=250, min_weight_fraction_leaf=0.25;, score=(train=0.822, test=0.819) tot

[CV 3/5] END max_depth=25, max_leaf_nodes=2, min_impurity_decrease=0.25, min_samples_leaf=1, min_samples_split=100, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  18.2s
[CV 4/5] END max_depth=25, max_leaf_nodes=2, min_impurity_decrease=0.25, min_samples_leaf=1, min_samples_split=100, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  17.9s
[CV 5/5] END max_depth=25, max_leaf_nodes=2, min_impurity_decrease=0.25, min_samples_leaf=1, min_samples_split=100, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  18.0s
[CV 1/5] END max_depth=25, max_leaf_nodes=10, min_impurity_decrease=0.0, min_samples_leaf=100, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.658) total time=  17.0s
[CV 2/5] END max_depth=25, max_leaf_nodes=10, min_impurity_decrease=0.0, min_samples_leaf=100, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.658) total time=  17.5s
[CV

[CV 4/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=1000, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.500, test=0.500) total time=  18.0s
[CV 5/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.25, min_samples_leaf=1000, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.500, test=0.500) total time=  18.2s
[CV 1/5] END max_depth=50, max_leaf_nodes=2, min_impurity_decrease=0.25, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  18.1s
[CV 2/5] END max_depth=50, max_leaf_nodes=2, min_impurity_decrease=0.25, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  18.0s
[CV 3/5] END max_depth=50, max_leaf_nodes=2, min_impurity_decrease=0.25, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time= 

[CV 5/5] END max_depth=12, max_leaf_nodes=5, min_impurity_decrease=0.0, min_samples_leaf=250, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.833, test=0.835) total time=  36.3s
[CV 1/5] END max_depth=75, max_leaf_nodes=2, min_impurity_decrease=0.75, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.0s
[CV 2/5] END max_depth=75, max_leaf_nodes=2, min_impurity_decrease=0.75, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.9s
[CV 3/5] END max_depth=75, max_leaf_nodes=2, min_impurity_decrease=0.75, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.6s
[CV 4/5] END max_depth=75, max_leaf_nodes=2, min_impurity_decrease=0.75, min_samples_leaf=100, min_samples_split=250, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  17.1s
[

...

[CV 5/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.75, min_samples_leaf=50, min_samples_split=250, min_weight_fraction_leaf=0.5;, score=(train=0.500, test=0.500) total time=  21.0s
[CV 1/5] END max_depth=12, max_leaf_nodes=5, min_impurity_decrease=0.0, min_samples_leaf=100, min_samples_split=50, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.658) total time=  20.8s
[CV 2/5] END max_depth=12, max_leaf_nodes=5, min_impurity_decrease=0.0, min_samples_leaf=100, min_samples_split=50, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.658) total time=  21.0s
[CV 3/5] END max_depth=12, max_leaf_nodes=5, min_impurity_decrease=0.0, min_samples_leaf=100, min_samples_split=50, min_weight_fraction_leaf=0.5;, score=(train=0.658, test=0.656) total time=  21.2s
[CV 4/5] END max_depth=12, max_leaf_nodes=5, min_impurity_decrease=0.0, min_samples_leaf=100, min_samples_split=50, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.657) total time=  20.6s
[CV 5/5]

[CV 2/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.5, min_samples_leaf=100, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  22.1s
[CV 3/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.5, min_samples_leaf=100, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  22.5s
[CV 4/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.5, min_samples_leaf=100, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  22.0s
[CV 5/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.5, min_samples_leaf=100, min_samples_split=1000, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  22.2s
[CV 1/5] END max_depth=12, max_leaf_nodes=3, min_impurity_decrease=0.75, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.500, test=0.500) total time= 

[CV 3/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.0, min_samples_leaf=1000, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.844, test=0.844) total time=  44.0s
[CV 4/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.0, min_samples_leaf=1000, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.845, test=0.843) total time=  43.7s
[CV 5/5] END max_depth=50, max_leaf_nodes=10, min_impurity_decrease=0.0, min_samples_leaf=1000, min_samples_split=100, min_weight_fraction_leaf=0.0;, score=(train=0.845, test=0.847) total time=  44.1s
[CV 1/5] END max_depth=50, max_leaf_nodes=5, min_impurity_decrease=0.25, min_samples_leaf=100, min_samples_split=2, min_weight_fraction_leaf=0.0;, score=(train=0.500, test=0.500) total time=  18.4s
[CV 2/5] END max_depth=50, max_leaf_nodes=5, min_impurity_decrease=0.25, min_samples_leaf=100, min_samples_split=2, min_weight_fraction_leaf=0.0;, score=(train=0.500, test=0.500) total time=  18.3s
[

[CV 4/5] END max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.657) total time=  17.3s
[CV 5/5] END max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=50, min_samples_split=100, min_weight_fraction_leaf=0.5;, score=(train=0.657, test=0.658) total time=  17.6s
[CV 1/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.75, min_samples_leaf=1000, min_samples_split=50, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  18.3s
[CV 2/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.75, min_samples_leaf=1000, min_samples_split=50, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total time=  18.3s
[CV 3/5] END max_depth=12, max_leaf_nodes=10, min_impurity_decrease=0.75, min_samples_leaf=1000, min_samples_split=50, min_weight_fraction_leaf=0.25;, score=(train=0.500, test=0.500) total t

RandomizedSearchCV(estimator=RandomForestClassifier(n_estimators=250,
                                                    random_state=42),
                   n_iter=250,
                   param_distributions={'max_depth': [None, 25, 50, 75, 12],
                                        'max_leaf_nodes': [None, 2, 3, 5, 10],
                                        'min_impurity_decrease': [0.0, 0.25,
                                                                  0.5, 0.75],
                                        'min_samples_leaf': [1, 50, 100, 250,
                                                             1000],
                                        'min_samples_split': [2, 50, 100, 250,
                                                              1000],
                                        'min_weight_fraction_leaf': [0.0, 0.25,
                                                                     0.5]},
                   return_train_score=True, verbose=3)

In [86]:
bestForest_ = GSCV.best_estimator_

In [87]:
bestForest_.score(X_train_sub, y_train_mid)

0.9564551753350882

In [88]:
print('accuracy:', accuracy_score(bestForest_.predict(X_test_sub), y_test_mid))
print('f1_score:', f1_score(bestForest_.predict(X_test_sub), y_test_mid))
print('roc_auc_score:', roc_auc_score(bestForest_.predict(X_test_sub), y_test_mid))
print('precision_score:', precision_score(bestForest_.predict(X_test_sub), y_test_mid))
print('recall_score:', recall_score(bestForest_.predict(X_test_sub), y_test_mid))
bias = 1 - accuracy_score(bestForest_.predict(X_train_sub), y_train_mid)
print('bias = ', bias)
variance = 1 - accuracy_score(bestForest_.predict(X_test_sub), y_test_mid) - bias
print('variance = ', variance)

accuracy: 0.9518
f1_score: 0.9520015933081061
roc_auc_score: 0.9519926110839103
precision_score: 0.9639040129058278
recall_score: 0.9403895337399174
bias =  0.04354482466491183
variance =  0.004655175335088191


In [89]:
rfeTree = RFECV(DecisionTreeClassifier(random_state = 42), min_features_to_select = 5)

In [90]:
rfeTree.fit(X_train_mid, y_train_mid)

RFECV(estimator=DecisionTreeClassifier(random_state=42),
      min_features_to_select=5)

In [91]:
rfeTree.ranking_

array([1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1])

In [92]:
X_train_sub_tree = rfeTree.transform(X_train_mid)
X_test_sub_tree = rfeTree.transform(X_test_mid)

In [93]:
TreeSub = DecisionTreeClassifier(random_state = 42)
TreeSub.fit(X_train_sub_tree, y_train_mid)
print(TreeSub.score(X_train_sub_tree, y_train_mid))
print(TreeSub.score(X_test_sub_tree, y_test_mid))

0.9930773652839551
0.9571


In [94]:
params = {
    'min_samples_split' : [2, 50, 100, 250, 1000],\
    'min_samples_leaf' : [1, 50, 100, 250, 1000], \
    'min_weight_fraction_leaf' : [0.0, 0.25, 0.5],\
    'max_leaf_nodes' : [None, 2, 3, 5, 10],\
    'min_impurity_decrease' : [0.0, 0.25, 0.5, 0.75],\
    'max_depth' : [None, 25, 50, 75, 12]
}

In [95]:
TGS = GridSearchCV(TreeSub, params)

In [96]:
TGS.fit(X_train_sub_tree, y_train_mid)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [None, 25, 50, 75, 12],
                         'max_leaf_nodes': [None, 2, 3, 5, 10],
                         'min_impurity_decrease': [0.0, 0.25, 0.5, 0.75],
                         'min_samples_leaf': [1, 50, 100, 250, 1000],
                         'min_samples_split': [2, 50, 100, 250, 1000],
                         'min_weight_fraction_leaf': [0.0, 0.25, 0.5]})

In [97]:
bestTree = TGS.best_estimator_

In [100]:
y_pred = bestTree.predict(X_test_sub_tree)
y_pred_train = bestTree.predict(X_train_sub_tree)
print('accuracy:', accuracy_score(y_test_mid, y_pred))
print('f1_score:', f1_score(y_test_mid, y_pred))
print('roc_auc_score:', roc_auc_score(y_test_mid, y_pred))
print('precision_score:', precision_score(y_test_mid, y_pred))
print('recall_score:', recall_score(y_test_mid, y_pred))
bias = 1 - accuracy_score(y_train_mid, y_pred_train)
print('bias = ', bias)
variance = 1 - accuracy_score(y_test_mid, y_pred) - bias
print('variance = ', variance)

accuracy: 0.9571
f1_score: 0.9565305502077212
roc_auc_score: 0.9570569325081418
precision_score: 0.9613034623217923
recall_score: 0.9518047993547086
bias =  0.006922634716044929
variance =  0.03597736528395512


In [113]:
from sklearn.ensemble import RandomForestClassifier

In [114]:
forest = RandomForestClassifier(random_state = 42)
forest.fit(X_train_mid, y_train_mid)

RandomForestClassifier(random_state=42)

In [117]:
rfeForest = RFECV(forest, min_features_to_select = 5, verbose = 1, step = 2, cv = 3)
rfeForest.fit(X_train_mid, y_train_mid)

Fitting estimator with 13 features.
Fitting estimator with 11 features.
Fitting estimator with 9 features.
Fitting estimator with 7 features.
Fitting estimator with 13 features.
Fitting estimator with 11 features.
Fitting estimator with 9 features.
Fitting estimator with 7 features.
Fitting estimator with 13 features.
Fitting estimator with 11 features.
Fitting estimator with 9 features.
Fitting estimator with 7 features.


RFECV(cv=3, estimator=RandomForestClassifier(random_state=42),
      min_features_to_select=5, step=2, verbose=1)

In [118]:
print(forest.score(X_train_mid, y_train_mid))
print(forest.score(X_test_mid, y_test_mid))
ModelMetrics(forest)

0.9930531602674655
0.9637
accuracy: 0.9637
f1_score: 0.9634109464771696
roc_auc_score: 0.9636956230591879
precision_score: 0.9637023593466425
recall_score: 0.9631197097944377
bias =  0.0069468397325345155
variance =  0.029353160267465483


In [119]:
X_train_forest_sub = rfeForest.transform(X_train_mid)
X_test_forest_sub = rfeForest.transform(X_test_mid)

In [120]:
forest = RandomForestClassifier(random_state = 42)
forest.fit(X_train_forest_sub, y_train_mid)
print(forest.score(X_train_forest_sub, y_train_mid))
print(forest.score(X_test_forest_sub, y_test_mid))

0.9930531602674655
0.9637


In [121]:
print('accuracy:', accuracy_score(forest.predict(rfeForest.transform(X_test)), y_test))
print('f1_score:', f1_score(forest.predict(rfeForest.transform(X_test)), y_test))
print('roc_auc_score:', roc_auc_score(forest.predict(rfeForest.transform(X_test)), y_test))
print('precision_score:', precision_score(forest.predict(rfeForest.transform(X_test)), y_test))
print('recall_score:', recall_score(forest.predict(rfeForest.transform(X_test)), y_test))
bias = 1 - accuracy_score(forest.predict(X_train_forest_sub), y_train_mid)
print('bias = ', bias)
variance = 1 - accuracy_score(forest.predict(rfeForest.transform(X_test)), y_test) - bias
print('variance = ', variance)

accuracy: 0.9061
f1_score: 0.603295310519645
roc_auc_score: 0.7350162656586102
precision_score: 0.7735644637053087
recall_score: 0.49445983379501385
bias =  0.0069468397325345155
variance =  0.08695316026746547


In [122]:
from sklearn.model_selection import RandomizedSearchCV

In [123]:
clf = RandomizedSearchCV(forest, params, verbose = 1, n_iter = 250, cv = 3)
clf.fit(X_train_forest_sub, y_train_mid)

Fitting 3 folds for each of 250 candidates, totalling 750 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=250,
                   param_distributions={'max_depth': [None, 25, 50, 75, 12],
                                        'max_leaf_nodes': [None, 2, 3, 5, 10],
                                        'min_impurity_decrease': [0.0, 0.25,
                                                                  0.5, 0.75],
                                        'min_samples_leaf': [1, 50, 100, 250,
                                                             1000],
                                        'min_samples_split': [2, 50, 100, 250,
                                                              1000],
                                        'min_weight_fraction_leaf': [0.0, 0.25,
                                                                     0.5]},
                   verbose=1)

In [124]:
clf.best_params_

{'min_weight_fraction_leaf': 0.0,
 'min_samples_split': 250,
 'min_samples_leaf': 50,
 'min_impurity_decrease': 0.0,
 'max_leaf_nodes': None,
 'max_depth': 50}

In [125]:
bestForest = clf.best_estimator_

In [126]:
print(bestForest.score(X_train_forest_sub, y_train_mid))
print(bestForest.score(X_test_forest_sub, y_test_mid))

0.9239478381894648
0.922


In [127]:
print('accuracy:', accuracy_score(bestForest.predict(rfeForest.transform(X_test)), y_test))
print('f1_score:', f1_score(bestForest.predict(rfeForest.transform(X_test)), y_test))
print('roc_auc_score:', roc_auc_score(bestForest.predict(rfeForest.transform(X_test)), y_test))
print('precision_score:', precision_score(bestForest.predict(rfeForest.transform(X_test)), y_test))
print('recall_score:', recall_score(bestForest.predict(rfeForest.transform(X_test)), y_test))
bias = 1 - accuracy_score(bestForest.predict(X_train_forest_sub), y_train_mid)
print('bias = ', bias)
variance = 1 - accuracy_score(bestForest.predict(rfeForest.transform(X_test)), y_test) - bias
print('variance = ', variance)

accuracy: 0.879
f1_score: 0.5736434108527132
roc_auc_score: 0.7057917589062684
precision_score: 0.8819068255687974
recall_score: 0.425065274151436
bias =  0.0760521618105352
variance =  0.044947838189464795


In [128]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
forest = RandomForestClassifier(random_state = 42, n_estimators = 500)
forest.fit(X_train_mid, y_train_mid)
print(forest.score(X_train_mid, y_train_mid))
print(forest.score(X_test_mid, y_test_mid))

0.9930773652839551
0.9636


In [130]:
ModelMetrics(forest)

accuracy: 0.9636
f1_score: 0.9633434038267874
roc_auc_score: 0.9635918152286643
precision_score: 0.9645089735833837
recall_score: 0.9621806477569905
bias =  0.006922634716044929
variance =  0.02947736528395506


In [131]:
from joblib import dump

In [132]:
dump(forest, 'forest.joblib')

['forest.joblib']